In [1]:
from imutils.object_detection import non_max_suppression
import numpy as np
import time
import cv2
import PIL
from PIL import Image
import IPython
from IPython.display import clear_output
import matplotlib
import matplotlib.pyplot as plt

try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
    from io import BytesIO

In [2]:
# USE This method instaed of cv2.imageShow
def showarray(a, fmt='jpeg'):
    a = np.uint8(a)
    f = BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

In [3]:
argpath = 'images/lebron_james.jpg.jpg'
argeast = 'frozen_east_text_detection.pb'
argmincon = 0.51

In [5]:
# load the input image and grab the image dimensions
image = cv2.imread(argpath)
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

orig = image.copy()
(H, W) = image.shape[:2]
print('original dimension :',(H, W))

# find optimum resize scale
# linear resize : H=32X+C / W=32Y+D

golden_ratio = (1 + 5 ** 0.5) / 2
if H < 1000 or W < 1000 : resize_ratio = 1
else : resize_ratio = golden_ratio
print(resize_ratio)
C = H%32; D=W%32
HminusC = H-C; WminusD = W-D
HminusC_ratiomod = (HminusC/resize_ratio)/32; WminusD_ratiomod = (WminusD/resize_ratio)/32

argw = 32*int(WminusD_ratiomod);argh = 32*int(HminusC_ratiomod)
print('resize dimension 32X:',argh,argw)

AttributeError: 'NoneType' object has no attribute 'copy'

In [6]:
# set the new width and height and then determine the ratio in change
# for both the width and height
(newW, newH) = (argw, argh)
rW = W / float(newW)
rH = H / float(newH)

# resize the image and grab the new image dimensions
image = cv2.resize(image, (newW, newH))
(H, W) = image.shape[:2]
print((H, W))
# define the two output layer names for the EAST detector model that
# we are interested -- the first is the output probabilities and the
# second can be used to derive the bounding box coordinates of text
layerNames = ["feature_fusion/Conv_7/Sigmoid","feature_fusion/concat_3"]

NameError: name 'argw' is not defined

In [ ]:
# find average color : Gaussian blur kernel odd --> Median on Median
# image = cv2.GaussianBlur(image, (3,3), 0)

# np.average() takes in an axis argument which finds the average across that axis. 
average_color_per_row = np.median(image, axis=0)

# find median across average per row
average_color = np.median(average_color_per_row, axis=0)

# convert back to RGB tuples
average_color = tuple(np.round(average_color,2))
print(average_color)

showarray(image)

In [ ]:
# load the pre-trained EAST text detector
print("[INFO] loading EAST text detector...")
net = cv2.dnn.readNet(argeast)

# construct a blob from the image and then perform a forward pass of
# the model to obtain the two output layer sets
blob = cv2.dnn.blobFromImage(image, 1.0, (W, H),average_color, swapRB=True, crop=False)
start = time.time()
net.setInput(blob)
(scores, geometry) = net.forward(layerNames)
end = time.time()

# show timing information on text prediction
print("[INFO] text detection took {:.6f} seconds".format(end - start))

# grab the number of rows and columns from the scores volume, then
# initialize our set of bounding box rectangles and corresponding
# confidence scores
(numRows, numCols) = scores.shape[2:4]
rects = []
confidences = []

In [ ]:
# loop over the number of rows
for y in range(0, numRows):
    # extract the scores (probabilities), followed by the geometrical
    # data used to derive potential bounding box coordinates that
    # surround text
    scoresData = scores[0, 0, y]
    xData0 = geometry[0, 0, y]
    xData1 = geometry[0, 1, y]
    xData2 = geometry[0, 2, y]
    xData3 = geometry[0, 3, y]
    anglesData = geometry[0, 4, y]

    # loop over the number of columns
    for x in range(0, numCols):
        # if our score does not have sufficient probability, ignore it
        if scoresData[x] < argmincon:
            continue

        # compute the offset factor as our resulting feature maps will
        # be 4x smaller than the input image
        (offsetX, offsetY) = (x * 4.0, y * 4.0)

        # extract the rotation angle for the prediction and then
        # compute the sin and cosine
        angle = anglesData[x]
        cos = np.cos(angle)
        sin = np.sin(angle)

        # use the geometry volume to derive the width and height of
        # the bounding box
        h = xData0[x] + xData2[x]
        w = xData1[x] + xData3[x]

        # compute both the starting and ending (x, y)-coordinates for
        # the text prediction bounding box
        endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
        endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
        startX = int(endX - w)
        startY = int(endY - h)

        # add the bounding box coordinates and probability score to
        # our respective lists
        rects.append((startX, startY, endX, endY))
        confidences.append(scoresData[x])

In [ ]:
# apply non-maxima suppression to suppress weak, overlapping bounding
# boxes
boxes = non_max_suppression(np.array(rects), probs=confidences)

# loop over the bounding boxes
for (startX, startY, endX, endY) in boxes:
    # scale the bounding box coordinates based on the respective
    # ratios
    startX = int(startX * rW)
    startY = int(startY * rH)
    endX = int(endX * rW)
    endY = int(endY * rH)

    # draw the bounding box on the image
    cv2.rectangle(orig, (startX, startY), (endX, endY), (0, 255, 0), 3)

In [ ]:
showarray(orig)
# Display the frame until new frame is available
clear_output(wait=True)